<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/189__B2B_Sales_Orchestrator_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# B2B Sales Orchestrator Agent - Scaffold Plan

**Status:** 📋 Planning Phase - Review Before Implementation  
**Purpose:** Build MVP sales orchestrator that researches companies, plans personalized outreach, and generates reports  
**Learning Focus:** Multi-source data orchestration, LLM-powered personalization, conditional routing (future), state management

---

## Overview

Build an orchestrator agent that takes a company name/URL as input, performs research, generates personalized outreach plans, and creates comprehensive lead research reports. This MVP focuses on the **research → planning → reporting** workflow without actual email/LinkedIn sending.

**Key Learning Goals:**
- Practice multi-source data collection (web search, company data)
- Learn LLM-powered personalization patterns
- Build orchestrator patterns for multi-stage workflows
- Create actionable, formatted reports

**MVP Scope (Incremental Approach):**
- ✅ Single company input (Target for testing)
- ✅ Real web research (Tavily API - user has key)
- ✅ **Fixed template for decision-makers** (not LLM-generated, focus on orchestration)
- ✅ **Fixed defaults for ICP scoring** (not configurable, get logic working first)
- ✅ **Dummy data for personalization** (get MVP working, then improve incrementally)
- ✅ Markdown reports as output
- ❌ No CRM integration (mock state only)
- ❌ No email/LinkedIn sending (just generate drafts)
- ❌ No real contact enrichment APIs (use placeholders)

**Development Philosophy:**
Start with templates/defaults/dummy data → Get orchestration working → Replace dummy data incrementally (one section at a time) → Test → Debug → Isolate issues → Move to next section

---

## What This Agent Does (MVP)

1. **Research** company background, industry, pain points, buying signals
2. **Plan** personalized outreach strategy (message drafts, channel selection, timing)
3. **Generate** comprehensive lead research report and outreach plan

**Input:** Company name and website URL (optional product/service description)  
**Output:** Lead research report (markdown) + Outreach plan (JSON + markdown)

---

## State Schema (Plain English)

The agent will track:

**Input Fields:**
- `company_name`: Company name to research (e.g., "Acme Corp")
- `company_website`: Optional website URL
- `product_service`: Optional description of what we're selling (for personalization)

**Research Data:**
- `company_research`: Raw research data from web search
  - Company overview (industry, size, revenue estimates, growth stage)
  - Recent news/articles (pain points, buying signals)
  - Technology stack (if available)
  - Job postings (hiring signals, needs)
  - Industry trends (market context)
- `decision_makers`: Mock/synthetic decision-maker data (for MVP)
  - Name, title, LinkedIn placeholder
  - Role in decision-making process
  - Contact preference (email vs LinkedIn)

**Analysis Results:**
- `company_profile`: Structured company profile
  - Industry, size, revenue estimate, growth stage
  - Key pain points (from research)
  - Buying signals (funding, hiring, expansion)
  - ICP fit score (0-100, based on criteria)
  - Technology alignment
- `pain_points`: Extracted pain points and challenges
- `buying_signals`: Identified buying signals (funding, hiring, news, etc.)
- `fit_assessment`: ICP fit analysis
  - Fit score (0-100)
  - Fit reasons (why it's a good/bad fit)
  - Priority level (high/medium/low)

**Outreach Planning:**
- `outreach_plan`: Personalized outreach strategy
  - Target contact (decision-maker)
  - Channel recommendation (email vs LinkedIn)
  - Timing recommendation (best day/time)
  - Value proposition (personalized based on research)
  - Message drafts (initial + follow-up sequence)
  - Personalization elements (company-specific hooks)

**Output:**
- `research_report`: Generated lead research report (markdown)
- `outreach_plan_markdown`: Formatted outreach plan (markdown)
- `report_file_paths`: Paths to saved report files

**Metadata:**
- `errors`: Any errors encountered
- `processing_time`: Time taken
- `research_sources`: List of sources used for research

---

## Node Design (Minimal Linear Flow for MVP)

Following your guide: **Start with minimal linear flow, add conditional routing later when needed.**

### Node 1: `goal_node` - Define Research Goal
**Purpose:** Set up the research and outreach planning framework

**What it does:**
- Define research goal (understand company, identify pain points, assess fit)
- Set up outreach planning objective (create personalized outreach strategy)
- Structure goal as dictionary with:
  - Research objectives (what to find out)
  - Outreach objectives (what to create)
  - Product/service context (for personalization)
  - ICP criteria (ideal customer profile match criteria)

**Input (State):**
- `company_name`
- `company_website` (optional)
- `product_service` (optional)

**Output (State):**
- `goal`: Research and outreach planning goal definition

**Logic:**
- Fixed structure (template-based, no LLM needed)
- Use standard research objectives
- Include product/service context if provided

---

### Node 2: `planning_node` - Create Research & Outreach Plan
**Purpose:** Define the research strategy and outreach planning approach

**What it does:**
- Create execution plan based on goal
- Define research steps (company overview, pain points, buying signals, decision-makers)
- Define outreach planning steps (personalization strategy, message crafting, channel selection)
- Structure plan as list of steps

**Input (State):**
- `goal`

**Output (State):**
- `plan`: Execution plan with research and outreach planning steps

**Logic:**
- Template-based plan (no LLM needed)
- Plan structure: step number, action, node responsible

---

### Node 3: `research_node` - Collect Company Data
**Purpose:** Research company using web search and extract structured insights

**What it does:**
- Perform web searches for:
  - Company overview (industry, size, revenue, growth stage)
  - Recent news/articles (pain points, challenges, initiatives)
  - Technology stack (if available)
  - Job postings (hiring signals, needs)
  - Industry trends (market context)
- Use multiple search queries (per framework-specific search strategy pattern)
- Collect data from multiple sources (Tavily, web search, etc.)
- Extract structured insights:
  - Company profile (industry, size, revenue estimate, growth stage)
  - Pain points (from news, articles, job postings)
  - Buying signals (funding, hiring, expansion, technology adoption)
  - Technology alignment (if available)

**Input (State):**
- `company_name`
- `company_website` (optional - helps with search)
- `goal` (research objectives)
- `plan` (research steps)

**Output (State):**
- `company_research`: Raw research data (search results, articles, etc.)
- `company_profile`: Structured company profile
- `pain_points`: Extracted pain points
- `buying_signals`: Identified buying signals
- `research_sources`: List of sources used

**Logic:**
- Web search API calls (Tavily, SerpAPI, or Bing)
- Multiple targeted queries (e.g., "{company} industry", "{company} challenges", "{company} funding")
- LLM-powered extraction (summarize search results, extract structured data)
- **Error handling:**
  - API failures → fail gracefully, log error, continue with available data
  - No results found → log warning, use partial data
  - Invalid responses → retry once, then fail gracefully

**Challenges:**
- Extracting structured data from unstructured web search results
- Identifying relevant pain points from news/articles
- Determining company size/revenue from public sources (may need estimates)
- Identifying buying signals from various sources

---

### Node 4: `analyze_node` - Analyze Fit & Generate Decision-Makers
**Purpose:** Assess ICP fit and generate mock decision-maker data

**What it does:**
- Analyze company profile against **fixed default ICP criteria**
- Calculate fit score (0-100) using **deterministic algorithm**
- Generate fit assessment (fit reasons, priority level)
- **Generate decision-makers using fixed template** (not LLM-generated for MVP):
  - Use template structure: VP/Director level roles
  - Create synthetic contact data (name, title, LinkedIn placeholder)
  - Mark clearly as "PLACEHOLDER" or "FAKE"
  - Recommend contact preference (email vs LinkedIn)

**Input (State):**
- `company_profile`
- `pain_points`
- `buying_signals`
- `goal` (ICP criteria)

**Output (State):**
- `fit_assessment`: ICP fit analysis (score, reasons, priority)
- `decision_makers`: Mock decision-maker data (synthetic for MVP)

**Logic:**
- Fit scoring algorithm (deterministic, based on **fixed default ICP criteria**)
- **Fixed template for decision-makers** (not LLM-generated for MVP):
  - Template: ["VP of Sales", "Director of Operations", "VP of Engineering"]
  - Generate 1-3 decision-makers using template
  - Mark clearly as "PLACEHOLDER" in output
- **Error handling:**
  - Missing company data → lower fit score, log warning
  - Use default template structure (no LLM calls needed for MVP)

**Notes:**
- **MVP:** Fixed template (focus on orchestration, not data quality)
- **Phase 2:** Replace with LLM-generated or real APIs
- Clearly mark all placeholder data in output

---

### Node 5: `outreach_plan_node` - Generate Personalized Outreach Plan
**Purpose:** Create personalized outreach strategy and message drafts

**What it does:**
- **MVP: Use dummy data for personalization** (get orchestration working first):
  - Use template message with company name insertion
  - Use dummy pain points and buying signals
  - Generate basic outreach plan structure
- **Phase 2: Replace with LLM-powered personalization**:
  - Analyze company research and pain points (real data)
  - Craft personalized value proposition (based on research findings)
  - Select best channel (email vs LinkedIn) based on decision-maker profile
  - Determine optimal timing (industry patterns, timezone)
  - Create initial message draft (personalized, value-focused, concise)
  - Create follow-up sequence (if no response)
- Include personalization elements (dummy for MVP, real for Phase 2):
  - Company-specific hooks (reference recent news, funding, initiatives)
  - Pain point references (address specific challenges)
  - Value proposition alignment (connect product/service to company needs)

**Input (State):**
- `company_profile`
- `pain_points`
- `buying_signals`
- `fit_assessment`
- `decision_makers`
- `product_service` (for personalization)
- `goal` (outreach objectives)

**Output (State):**
- `outreach_plan`: Personalized outreach strategy
  - Target contact (decision-maker)
  - Channel recommendation
  - Timing recommendation
  - Value proposition
  - Message drafts (initial + follow-ups)
  - Personalization elements

**Logic:**
- **MVP: Template-based with dummy data** (company name insertion only)
- **Phase 2: LLM-powered message generation** (personalized based on real research)
- Template-based structure (ensure consistent format)
- **Error handling:**
  - LLM API failures → retry once, then fail gracefully (Phase 2)
  - Invalid responses → log error, use fallback template

**Development Strategy:**
- **MVP:** Get basic outreach plan structure working (dummy data)
- **Phase 2:** Replace dummy data with real research → test → debug → isolate issues
- **Phase 3:** Improve message quality (LLM personalization) → test → debug

---

### Node 6: `report_node` - Generate Lead Research Report
**Purpose:** Generate comprehensive markdown reports (research report + outreach plan)

**What it does:**
- Generate lead research report using Jinja2 template:
  - Company overview
  - Key decision-makers (marked as placeholder for MVP)
  - Pain points and challenges
  - Buying signals
  - Fit assessment
  - Recommended outreach approach
- Generate outreach plan document:
  - Outreach strategy summary
  - Personalized message drafts
  - Follow-up sequence
  - Channel and timing recommendations
- Save reports to files (e.g., `lead_research_<company>_<timestamp>.md`)

**Input (State):**
- `company_profile`
- `pain_points`
- `buying_signals`
- `fit_assessment`
- `decision_makers`
- `outreach_plan`
- `research_sources`

**Output (State):**
- `research_report`: Generated lead research report (markdown)
- `outreach_plan_markdown`: Formatted outreach plan (markdown)
- `report_file_paths`: Paths to saved report files

**Logic:**
- Jinja2 template rendering
- Format markdown with proper headers, sections, tables
- Include clear indicators for placeholder/synthetic data (decision-makers)
- **Error handling:** Template render fail → fail immediately (can't produce output without template)

---

## Graph Wiring (MVP: Linear Flow)

**Start with linear flow only:**

```
goal → planning → research → analyze → outreach_plan → report → END
```

**Rationale:** MVP doesn't need conditional routing initially. All companies go through the same research → planning → reporting pipeline. We can add conditional routing later if we want to:
- Skip certain research steps for known companies
- Route to different outreach strategies based on fit score
- Handle batch processing (multiple companies)

**Later Enhancement (Not MVP):**
- After `analyze_node`: if fit score < 50, route to "low priority" report format
- After `research_node`: if insufficient data, route to enhanced research or flag for manual review
- Batch processing: if directory/CSV input, route to batch processing subgraph

But for MVP: **Linear flow is sufficient.**

---

## Error Handling Strategy Matrix

| Error Type | Strategy | Example |
|------------|----------|---------|
| **Company name not found** | Fail gracefully | Log warning, continue with partial data, note in report |
| **Web search API failure** | Retry once, then fail gracefully | Log error, use available data, continue with partial results |
| **No research results** | Log warning, continue | Use partial data, note data gaps in report |
| **LLM API failure** | Retry once, then fail gracefully | Log error, use fallback template, continue |
| **Invalid LLM response** | Retry once, then fail gracefully | Log error, use fallback, continue |
| **Template render fail** | Fail immediately | Can't produce output without template |
| **File write fail** | Fail immediately | Can't save reports |

**Principle:** Fail fast for output issues; fail gracefully for research/analysis issues (continue with partial data, log warnings).

---

## Data Sources & Research Strategy

### Web Search Queries (Framework-Specific Pattern)

**Research Strategy:**
```python
RESEARCH_QUERIES = {
    "company_overview": [
        f"{company_name} company overview",
        f"{company_name} industry size revenue",
        f"{company_name} growth stage funding"
    ],
    "pain_points": [
        f"{company_name} challenges problems",
        f"{company_name} pain points",
        f"{company_name} recent news struggles"
    ],
    "buying_signals": [
        f"{company_name} funding raised",
        f"{company_name} hiring jobs",
        f"{company_name} expansion growth",
        f"{company_name} technology adoption"
    ],
    "technology": [
        f"{company_name} technology stack",
        f"{company_name} software tools",
        f"{company_name} infrastructure"
    ]
}
```

**Data Sources:**
- **Primary:** Tavily API (web search) or SerpAPI/Google Search
- **Secondary:** Company website (if provided)
- **Future:** Crunchbase, Pitchbook (if API access available)

**Research Collection:**
- Execute multiple queries per category
- Collect top N results per query
- Aggregate and deduplicate results
- Extract structured data using LLM

---

## ICP Fit Scoring (Deterministic - Fixed Defaults for MVP)

**ICP Criteria (Fixed Defaults - MVP):**
- Company size (employee count): 100-1000 employees = high fit (20 points)
- Industry: Retail/Technology = high fit (20 points)
- Growth stage: Established = medium fit (15 points)
- Technology alignment: Using similar tools = high fit (20 points)
- Pain points: Match our solution = high fit (25 points)

**Scoring Formula:**
- Each criterion contributes points (fixed for MVP)
- Total score: 0-100
- Priority levels:
  - High: 70-100
  - Medium: 40-69
  - Low: 0-39

**MVP Strategy:**
- Use fixed defaults to get scoring logic working
- **Phase 2:** Make criteria configurable (via goal/config)
- **Phase 3:** Add more sophisticated scoring (ML models, etc.)

---

## Folder Structure

```
project_root/
├── agents/
│   └── b2b_sales_orchestrator_agent.py    # LangGraph workflow (after smoke test)
├── nodes/
│   ├── __init__.py
│   ├── goal_node.py                       # Node 1: Define research goal
│   ├── planning_node.py                   # Node 2: Create research plan
│   ├── research_node.py                   # Node 3: Collect company data
│   ├── analyze_node.py                    # Node 4: Analyze fit & generate decision-makers
│   ├── outreach_plan_node.py              # Node 5: Generate personalized outreach plan
│   └── report_node.py                     # Node 6: Generate reports
├── prompts/
│   ├── __init__.py
│   ├── base_analyzer.py                   # Base prompt class (reuse existing)
│   ├── research_prompt.py                 # Research extraction prompt
│   └── outreach_prompt.py                 # Outreach planning prompt
├── templates/
│   ├── lead_research_report.md.j2         # Jinja2 template for research report
│   └── outreach_plan.md.j2                # Jinja2 template for outreach plan
├── utils/
│   ├── __init__.py
│   ├── web_search.py                      # Web search utilities (Tavily/SerpAPI)
│   ├── research_parser.py                 # Research data extraction utilities
│   └── validators.py                      # Data validation utilities (reuse existing)
├── tests/
│   ├── test_mvp_runner.py                 # ⭐ Smoke test (create first)
│   └── test_sales_orchestrator.py         # Integration test (after wiring)
├── config.py                              # State schema (SalesOrchestratorState TypedDict) + AgentConfig
├── requirements.txt
└── sales_reports/                         # Output directory for reports
```

---

## Implementation Order (Following Your Guide)

1. **Goal node** (simplest, defines structure) - Fixed logic, no dependencies
2. **Planning node** (uses goal) - Template-based, depends on goal structure
3. **Research node** (most complex, depends on setup) - Web search, LLM extraction
4. **Analyze node** (scoring logic, depends on research) - Fit assessment, mock data generation
5. **Outreach plan node** (LLM-powered, depends on analyze) - Personalized message generation
6. **Report node** (formats output) - Template rendering, file saving

**Why this order:** Build from simplest → most complex, test each before dependencies.

---

## Testing Strategy

### ⭐ Smoke Test First (Before LangGraph)
Create `test_mvp_runner.py` that:
- Manually calls nodes in sequence
- Tests with sample company name (e.g., "Zoom", "Notion", "Stripe")
- Verifies state contracts (what each node reads/writes)
- Catches 90% of contract issues before graph complexity

**Example:**
```python
def test_linear_flow():
    state = {
        "company_name": "Zoom",
        "company_website": "https://zoom.us",
        "product_service": "AI-driven sales analytics platform",
        "errors": []
    }
    
    state = goal_node(state)
    assert "goal" in state
    
    state = planning_node(state)
    assert "plan" in state
    
    state = research_node(state)
    assert "company_profile" in state
    
    state = analyze_node(state)
    assert "fit_assessment" in state
    
    state = outreach_plan_node(state)
    assert "outreach_plan" in state
    
    state = report_node(state)
    assert "research_report" in state
    
    print("✅ All nodes passed smoke test!")
```

### Unit Tests
- Test web search utilities with real API calls (mock responses)
- Test research parser with sample search results
- Test fit scoring algorithm with known inputs
- Test outreach message generation (personalization logic)

### Integration Tests
- Test full workflow with real company names (Zoom, Notion, Stripe)
- Test error handling (API failures, no results, etc.)
- Validate output reports (markdown format, completeness)

---

## Key Challenges & Solutions

### Challenge 1: Extracting Structured Data from Web Search
**Problem:** Web search returns unstructured text, need to extract structured company profile.

**Solution:**
- Use LLM to extract structured data from search results
- Define clear extraction schema (company profile, pain points, buying signals)
- Use Pydantic schemas for validation
- Retry logic if extraction fails

### Challenge 2: Generating Personalized Messages
**Problem:** Need truly personalized messages, not generic templates.

**Solution:**
- Use LLM with company research context
- Include specific pain points and buying signals in prompt
- Generate company-specific hooks (reference recent news, funding)
- Validate personalization (check for company name, specific references)

### Challenge 3: Mock Decision-Maker Data
**Problem:** Need realistic but clearly synthetic decision-makers.

**Solution (MVP):**
- Use **fixed template** (not LLM-generated)
- Template: ["VP of Sales", "Director of Operations", "VP of Engineering"]
- Generate 1-3 decision-makers using template
- Mark clearly as "PLACEHOLDER" in output
- **Phase 2:** Replace with LLM-generated or real APIs

### Challenge 4: ICP Fit Scoring
**Problem:** Need objective fit scoring criteria.

**Solution:**
- Define clear ICP criteria upfront
- Use deterministic scoring algorithm
- Make criteria configurable (via goal or config)
- Document scoring rationale in report

---

## Success Criteria

### MVP Success Criteria
- ✅ Successfully researches real companies using web search
- ✅ Extracts structured company profile, pain points, buying signals
- ✅ Calculates ICP fit score
- ✅ Generates personalized outreach messages
- ✅ Creates comprehensive lead research reports
- ✅ Handles API failures gracefully (continues with partial data)

### Quality Gates
- Smoke test passes (all nodes execute in sequence)
- Unit tests pass (web search, research parser, fit scorer)
- Integration test passes (full workflow with real company)
- Reports render correctly from templates
- Personalization validated (messages contain company-specific elements)
- Error handling works for common failure modes

---

## Future Enhancements (Not MVP)

1. **Conditional Routing:** Route based on fit score (high fit → proposal, low fit → nurture)
2. **Batch Processing:** Support CSV input with multiple companies
3. **Real Contact Enrichment:** Integrate Apollo, Clearbit, or Clay APIs
4. **Response Handling:** Analyze email/LinkedIn responses and route accordingly
5. **CRM Integration:** Sync with HubSpot, Salesforce
6. **Follow-up Sequences:** Automated follow-up tracking and sending
7. **Lead Scoring:** More sophisticated scoring with ML models
8. **Multi-Channel Outreach:** Actual email/LinkedIn sending

---

## Decisions Made ✅

1. **Input Format:** ✅ Single company name (Target for MVP testing)
2. **Web Search API:** ✅ Tavily (user has API key)
3. **Decision-Maker Generation:** ✅ Fixed template (not LLM-generated for MVP)
4. **ICP Criteria:** ✅ Fixed defaults (not configurable for MVP)
5. **Message Personalization:** ✅ Dummy data initially, replace incrementally

**Development Strategy:**
- Phase 1: MVP with templates/defaults/dummy data → Get orchestration working
- Phase 2: Replace dummy data incrementally → Test → Debug → Isolate → Next section

---

## Next Steps ✅

1. ✅ Review and refine scaffold plan
2. ✅ Create `PROJECT_REQUIREMENTS.md` entry
3. ⏭️ Begin implementation following the implementation order:
   - Create state schema in `config.py`
   - Implement nodes (goal → planning → research → analyze → outreach → report)
   - Create smoke test runner
   - Wire nodes into LangGraph
4. ⏭️ Test with Target company
5. ⏭️ Iterate on quality improvements (incremental approach)

---

*End of Scaffold Plan*



 # B2B Sales Orchestrator Agent - Project Requirements

**Project:** B2B Sales Orchestrator Agent (MVP)  
**Status:** Planning → Implementation  
**Last Updated:** Initial setup

---

## Quick Reference

| Setting | Value |
|---------|-------|
| **LLM Model** | `gpt-4o-mini` (default) |
| **Temperature** | `0.3` |
| **API Keys Location** | `API_KEYS.env` |
| **Output Directory** | `sales_reports/` |
| **Test Company** | Target (for MVP testing) |

---

## Project Overview

Build an MVP sales orchestrator agent that:
1. Researches companies using web search (Tavily)
2. Generates personalized outreach plans
3. Creates comprehensive lead research reports

**MVP Philosophy:** Start with templates/defaults/dummy data, get orchestration working, then improve quality incrementally.

---

## Input & Output

### Input Format
- **Single company name** (for MVP)
- **Optional:** Company website URL
- **Optional:** Product/service description (for personalization)

**MVP Test Input:**
```python
{
    "company_name": "Target",
    "company_website": "https://target.com",
    "product_service": "AI-driven sales analytics platform"  # Optional
}
```

### Output Format
- **Lead Research Report** (markdown) - Saved to `sales_reports/`
- **Outreach Plan** (markdown) - Saved to `sales_reports/`

---

## Data Sources & APIs

### Web Research
- **Primary:** Tavily API (web search)
- **API Key:** `TAVILY_API_KEY` in `API_KEYS.env`
- **Strategy:** Multiple targeted queries per company

### Decision-Makers (MVP)
- **Approach:** Fixed template (not LLM-generated)
- **Format:** Synthetic/placeholder data
- **Marking:** Clearly labeled as "PLACEHOLDER" or "FAKE" in output
- **Future:** Replace with real LinkedIn/contact enrichment APIs

### ICP Scoring (MVP)
- **Approach:** Fixed defaults (not configurable)
- **Scoring:** Deterministic algorithm with default criteria
- **Future:** Make configurable via goal/config

### Personalization (MVP)
- **Approach:** Dummy data initially, then replace incrementally
- **Strategy:** Get MVP working, then improve each section:
  1. Get basic orchestration working
  2. Replace dummy data in research → test
  3. Replace dummy data in outreach → test
  4. Replace dummy data in reports → test
- **Future:** Full LLM-powered personalization based on research

---

## Development Approach

### Phase 1: MVP with Templates/Dummy Data
**Goal:** Get orchestration working end-to-end

- Use fixed templates for decision-makers
- Use default ICP criteria
- Use dummy data for personalization
- Focus on: node execution, state management, graph wiring, error handling

### Phase 2: Incremental Quality Improvements
**Goal:** Replace dummy data with real data, one section at a time

**Order:**
1. **Research quality** → Real web search extraction
2. **Outreach personalization** → Real LLM-generated messages
3. **Fit scoring** → Real ICP criteria and scoring
4. **Decision-makers** → Real LinkedIn/contact data (if APIs available)

**Strategy:** Replace → Test → Debug → Isolate issues → Move to next section

---

## Technical Decisions

### Web Search API
- **Choice:** Tavily
- **Rationale:** User has API key, good for research use cases
- **Location:** API key in `API_KEYS.env`

### Decision-Maker Generation
- **MVP:** Fixed template (not LLM-generated)
- **Rationale:** Focus on orchestration first, not data quality
- **Future:** LLM-generated or real APIs

### ICP Scoring
- **MVP:** Fixed defaults
- **Rationale:** Get scoring logic working, then make it smart
- **Future:** Configurable criteria

### Personalization
- **MVP:** Dummy data
- **Rationale:** Incremental approach - test orchestration, then improve quality
- **Future:** Full LLM personalization based on research

---

## Code Patterns

### State Schema
- Use `TypedDict` for type safety
- Keep state flat when possible
- Document field purpose with comments

### Error Handling
- **API failures:** Retry once, then fail gracefully
- **Missing data:** Log warnings, continue with partial data
- **Template failures:** Fail immediately (can't produce output)

### Testing Strategy
- **Smoke test first:** Manual node execution before LangGraph wiring
- **Unit tests:** Web search, research parser, fit scorer
- **Integration tests:** Full workflow with real company (Target)

---

## Folder Structure

```
project_root/
├── agents/
│   └── b2b_sales_orchestrator_agent.py
├── nodes/
│   ├── goal_node.py
│   ├── planning_node.py
│   ├── research_node.py
│   ├── analyze_node.py
│   ├── outreach_plan_node.py
│   └── report_node.py
├── prompts/
│   ├── base_analyzer.py (reuse existing)
│   ├── research_prompt.py
│   └── outreach_prompt.py
├── templates/
│   ├── lead_research_report.md.j2
│   └── outreach_plan.md.j2
├── utils/
│   ├── web_search.py (Tavily integration)
│   ├── research_parser.py
│   └── validators.py (reuse existing)
├── tests/
│   ├── test_mvp_runner.py (smoke test)
│   └── test_sales_orchestrator.py
├── config.py
└── sales_reports/ (output directory)
```

---

## Success Criteria (MVP)

### Functional Requirements
- ✅ Successfully researches Target using Tavily
- ✅ Extracts basic company profile (industry, size estimates)
- ✅ Generates outreach plan with template decision-makers
- ✅ Calculates ICP fit score using defaults
- ✅ Creates markdown reports (research + outreach plan)
- ✅ Handles API failures gracefully

### Quality Requirements
- ✅ All nodes execute in sequence (smoke test passes)
- ✅ State contracts work correctly (each node reads/writes expected fields)
- ✅ Reports render from templates without errors
- ✅ Error handling works for common failure modes

---

## Next Steps

1. ✅ Review scaffold plan
2. ✅ Create PROJECT_REQUIREMENTS.md (this file)
3. ⏭️ Create state schema in `config.py`
4. ⏭️ Implement nodes (goal → planning → research → analyze → outreach → report)
5. ⏭️ Create smoke test runner
6. ⏭️ Wire nodes into LangGraph
7. ⏭️ Test with Target company
8. ⏭️ Iterate on quality improvements

---

## Notes

- **Tavily API Key:** Add `TAVILY_API_KEY` to `API_KEYS.env` if not already present
- **Test Company:** Use Target for MVP testing
- **Incremental Approach:** Focus on orchestration first, then improve data quality section by section

---

*This document will be updated as we build and learn.*



# B2B Sales Orchestrator Agent

In [ ]:
"""B2B Sales Orchestrator Agent - LangGraph workflow"""

import logging
from langgraph.graph import StateGraph, END
from config import SalesOrchestratorState
from nodes import (
    goal_node,
    planning_node,
    research_node,
    analyze_node,
    outreach_plan_node,
    report_node
)

logger = logging.getLogger(__name__)


def create_agent() -> StateGraph:
    """Create and compile the B2B Sales Orchestrator agent workflow

    Returns:
        Compiled LangGraph agent ready for execution
    """
    # Create StateGraph with our state schema
    workflow = StateGraph(SalesOrchestratorState)

    # Add all nodes
    workflow.add_node("goal", goal_node)
    workflow.add_node("planning", planning_node)
    workflow.add_node("research", research_node)
    workflow.add_node("analyze", analyze_node)
    workflow.add_node("outreach_plan", outreach_plan_node)
    workflow.add_node("report", report_node)

    # Linear flow: goal → planning → research → analyze → outreach_plan → report → END
    workflow.add_edge("goal", "planning")
    workflow.add_edge("planning", "research")
    workflow.add_edge("research", "analyze")
    workflow.add_edge("analyze", "outreach_plan")
    workflow.add_edge("outreach_plan", "report")
    workflow.add_edge("report", END)

    # Set entry point
    workflow.set_entry_point("goal")

    # Compile and return
    agent = workflow.compile()
    logger.info("✅ B2B Sales Orchestrator agent compiled successfully")

    return agent


def run_agent(company_name: str, company_website: str = None,
              product_service: str = None) -> SalesOrchestratorState:
    """Run the sales orchestrator agent for a company

    Args:
        company_name: Company name to research
        company_website: Optional website URL
        product_service: Optional product/service description

    Returns:
        Final state with research report and outreach plan
    """
    # Create agent
    agent = create_agent()

    # Initialize state
    initial_state: SalesOrchestratorState = {
        "company_name": company_name,
        "company_website": company_website,
        "product_service": product_service,
        "errors": []
    }

    # Run agent
    logger.info(f"🚀 Starting sales orchestrator for {company_name}...")
    final_state = agent.invoke(initial_state)

    logger.info(f"✅ Agent completed for {company_name}")
    if final_state.get("errors"):
        logger.warning(f"⚠️  Errors encountered: {len(final_state['errors'])}")

    return final_state


if __name__ == "__main__":
    """Run agent directly from command line"""
    import sys

    # Set up logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(levelname)s: %(message)s'
    )

    # Get company name from command line or use default
    if len(sys.argv) > 1:
        company_name = sys.argv[1]
        company_website = sys.argv[2] if len(sys.argv) > 2 else None
        product_service = sys.argv[3] if len(sys.argv) > 3 else None
    else:
        # Default to Target for testing
        company_name = "Target"
        company_website = "https://target.com"
        product_service = "AI-driven sales analytics platform"

    # Run agent
    result = run_agent(company_name, company_website, product_service)

    # Print summary
    print("\n" + "=" * 60)
    print("📊 Agent Execution Summary")
    print("=" * 60)
    print(f"Company: {result.get('company_name', 'Unknown')}")
    print(f"Fit Score: {result.get('fit_assessment', {}).get('fit_score', 'N/A')}/100")
    print(f"Priority: {result.get('fit_assessment', {}).get('priority_level', 'N/A')}")
    print(f"Reports Generated: {len(result.get('report_file_paths', {}))}")

    if result.get("report_file_paths"):
        print("\n📄 Report Files:")
        for report_type, path in result["report_file_paths"].items():
            print(f"  {report_type}: {path}")

    if result.get("errors"):
        print(f"\n⚠️  Errors: {len(result['errors'])}")
        for error in result["errors"]:
            print(f"  - {error}")

    print("=" * 60)



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_012 % python tests/test_sales_orchestrator.py
============================================================
🧪 Integration Test: B2B Sales Orchestrator Agent (LangGraph)
============================================================

📥 Initial State:
  Company: Target
  Website: https://target.com
  Product: AI-driven sales analytics platform

------------------------------------------------------------
Running LangGraph workflow...
------------------------------------------------------------
INFO: ✅ B2B Sales Orchestrator agent compiled successfully
INFO: 🎯 Defining research and outreach planning goal...
INFO: ✅ Goal defined for Target
INFO: 📋 Creating execution plan...
INFO: ✅ Execution plan created
INFO: 🔍 Researching company using web search...
INFO: Executing 4 search queries for Target...
INFO: Searching: Target company overview industry size revenue
INFO:   Found 3 results for 'company overview industry size revenue'
INFO: Searching: Target recent news challenges problems
INFO:   Found 3 results for 'recent news challenges problems'
INFO: Searching: Target funding hiring expansion growth
INFO:   Found 3 results for 'funding hiring expansion growth'
INFO: Searching: Target technology stack software tools
INFO:   Found 3 results for 'technology stack software tools'
INFO: Calling LLM to extract structured data...
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: ✅ Successfully extracted structured data from search results
INFO: ✅ Research complete
INFO:    Company profile: 5 fields
INFO:    Pain points: 5 found
INFO:    Buying signals: 2 found
INFO:    Sources: 12 unique
INFO: 🔍 Analyzing company fit and generating decision-makers...
INFO: ✅ Fit analysis complete: Score 74/100, Priority: high
INFO: ✅ Generated 3 decision-makers (template-based)
INFO: 📧 Generating personalized outreach plan...
INFO: ✅ Outreach plan generated
INFO:    Target: PLACEHOLDER - Sales 1
INFO:    Channel: linkedin
INFO:    Messages: 3 drafts
INFO: 📄 Generating lead research report and outreach plan...
INFO: Rendering research report template...
INFO: Rendering outreach plan template...
INFO: Saving research report to lead_research_Target_20251103_171551.md...
INFO: Saving outreach plan to outreach_plan_Target_20251103_171551.md...
INFO: ✅ Reports generated successfully
INFO:    Research report: /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_012/sales_reports/lead_research_Target_20251103_171551.md
INFO:    Outreach plan: /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_012/sales_reports/outreach_plan_Target_20251103_171551.md

------------------------------------------------------------
Verifying final state...
------------------------------------------------------------
✅ All required fields present in final state

============================================================
✅ Integration test passed!
   Company: Target
   Fit Score: 74/100
   Priority: high
   Pain Points: 5
   Buying Signals: 2
   Reports: 2
   Errors: 0

📄 Generated Reports:
   research_report: /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_012/sales_reports/lead_research_Target_20251103_171551.md
   outreach_plan: /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_012/sales_reports/outreach_plan_Target_20251103_171551.md
============================================================

============================================================
🧪 Testing run_agent() convenience function
============================================================
INFO: ✅ B2B Sales Orchestrator agent compiled successfully
INFO: 🚀 Starting sales orchestrator for Target...
INFO: 🎯 Defining research and outreach planning goal...
INFO: ✅ Goal defined for Target
INFO: 📋 Creating execution plan...
INFO: ✅ Execution plan created
INFO: 🔍 Researching company using web search...
INFO: Executing 4 search queries for Target...
INFO: Searching: Target company overview industry size revenue
INFO:   Found 3 results for 'company overview industry size revenue'
INFO: Searching: Target recent news challenges problems
INFO:   Found 3 results for 'recent news challenges problems'
INFO: Searching: Target funding hiring expansion growth
INFO:   Found 3 results for 'funding hiring expansion growth'
INFO: Searching: Target technology stack software tools
INFO:   Found 3 results for 'technology stack software tools'
INFO: Calling LLM to extract structured data...
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: ✅ Successfully extracted structured data from search results
INFO: ✅ Research complete
INFO:    Company profile: 5 fields
INFO:    Pain points: 4 found
INFO:    Buying signals: 0 found
INFO:    Sources: 12 unique
INFO: 🔍 Analyzing company fit and generating decision-makers...
INFO: ✅ Fit analysis complete: Score 74/100, Priority: high
INFO: ✅ Generated 3 decision-makers (template-based)
INFO: 📧 Generating personalized outreach plan...
INFO: ✅ Outreach plan generated
INFO:    Target: PLACEHOLDER - Sales 1
INFO:    Channel: linkedin
INFO:    Messages: 3 drafts
INFO: 📄 Generating lead research report and outreach plan...
INFO: Rendering research report template...
INFO: Rendering outreach plan template...
INFO: Saving research report to lead_research_Target_20251103_171559.md...
INFO: Saving outreach plan to outreach_plan_Target_20251103_171559.md...
INFO: ✅ Reports generated successfully
INFO:    Research report: /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_012/sales_reports/lead_research_Target_20251103_171559.md
INFO:    Outreach plan: /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_012/sales_reports/outreach_plan_Target_20251103_171559.md
INFO: ✅ Agent completed for Target
✅ run_agent() function works correctly
(.venv) micahshull@Micahs-iMac LG_Cursor_012 %

LangGraph workflow is working end-to-end.

## MVP complete

The integration test passed:
- All 6 nodes executed in sequence via LangGraph
- Real Tavily research (12 sources, 5 pain points, 2 buying signals)
- LLM extraction working
- Fit score: 74/100 (high priority)
- Reports generated successfully
- Zero errors

## What you've built

A B2B Sales Orchestrator Agent that:
1. Researches companies using Tavily web search
2. Extracts structured data with LLM
3. Calculates ICP fit scores
4. Generates decision-makers (template-based for MVP)
5. Creates personalized outreach plans
6. Produces markdown reports



# B2B Sales Orchestrator Agent - Build Reflection

**Date:** Build completed successfully  
**Agent:** B2B Sales Orchestrator (6-node linear workflow)  
**Status:** MVP complete, fully functional

---

## What Worked Exceptionally Well

### 1. **Scaffold Planning in Plain English First** ⭐⭐⭐
**What:** Created comprehensive scaffold plan before coding  
**Impact:**
- Avoided architectural mistakes
- Clear understanding of node responsibilities
- Defined state schema upfront
- Made implementation decisions explicit (templates, dummy data, MVP approach)

**Why it worked:**
- Forced us to think through the entire flow before coding
- Caught potential issues early (like "should we use templates or LLM for decision-makers?")
- Made the conversation about architecture, not code syntax

**Example:** We decided on "fixed template for decision-makers" vs "LLM-generated" upfront, which saved time and kept focus on orchestration.

---

### 2. **Incremental Dummy Data Approach** ⭐⭐⭐
**What:** Started with templates/defaults/dummy data, then replaced incrementally  
**Impact:**
- Got orchestration working end-to-end quickly
- Could test each node independently
- Isolated issues easily (knew exactly which node had problems)
- Built confidence progressively

**Why it worked:**
- Testing orchestration separately from data quality
- Could verify state contracts work before adding complexity
- Made debugging easier (if orchestration breaks, we knew it wasn't the data)

**Example:** We got all 6 nodes working with dummy data, THEN added real Tavily research. When research worked, we knew the issue wasn't in orchestration.

---

### 3. **Smoke Test Pattern (Before LangGraph)** ⭐⭐⭐
**What:** Tested nodes manually in sequence before wiring into LangGraph  
**Impact:**
- Caught import path issues immediately
- Verified state contracts work correctly
- Tested each node independently
- Caught 90% of issues before graph complexity

**Why it worked:**
- Simpler debugging (just Python function calls, not graph execution)
- Faster iteration (no need to recompile graph)
- Clear visibility into what each node does

**Example:** When we tested `research_node`, we immediately saw Tavily API key was missing. Caught it before wiring into LangGraph.

---

### 4. **Implementation Order (Simplest → Most Complex)** ⭐⭐
**What:** Implemented nodes in order: goal → planning → analyze → outreach → report → research  
**Impact:**
- Built momentum (easy wins first)
- Each node tested before dependencies
- Natural progression of complexity

**Why it worked:**
- Goal/planning nodes were simple (template-based, no APIs)
- Could test immediately
- Research node (most complex) came last, when we had confidence

**Note:** We actually did: goal → planning → analyze → outreach → report → research (research was last because it needed Tavily setup). This still worked because research doesn't depend on other nodes.

---

### 5. **State Schema Defined Upfront** ⭐⭐
**What:** Created complete TypedDict with all fields before implementing nodes  
**Impact:**
- Clear contract for each node (what it reads/writes)
- Type safety from the start
- Documented field purposes in comments

**Why it worked:**
- Prevented state schema mismatches
- Each node knew exactly what fields to expect
- Made it easy to see data flow

---

### 6. **PROJECT_REQUIREMENTS.md Pattern** ⭐⭐
**What:** Centralized all project decisions in one place  
**Impact:**
- Avoided repeated questions
- Clear "Quick Reference" for decisions
- Standard patterns documented

**Why it worked:**
- Single source of truth
- Fast decisions (look it up, don't ask)
- Consistent patterns across the build

---

## What Was Helpful But Could Be Improved

### 1. **Folder Structure Timing**
**What:** Guide mentioned folder structure, but we created it mid-way  
**Impact:**
- Had to reorganize files slightly
- Slightly unclear where files should go initially

**Improvement:** Create folder structure FIRST (step 3), before any code. Make it a concrete checklist item.

---

### 2. **Dependency Management**
**What:** We added `tavily-python` to requirements.txt, but didn't install until later  
**Impact:**
- Could have caught import errors earlier
- Had to remember to install dependencies

**Improvement:** Add a checklist item: "Install all dependencies from requirements.txt before testing"

---

### 3. **Virtual Environment Awareness**
**What:** User wasn't sure if `(.venv)` meant they were in venv  
**Impact:**
- Brief confusion about where packages were installed
- Could have installed globally by mistake

**Improvement:** Add explicit note: "`(.venv)` in prompt = virtual environment active. Always check with `which python` if unsure."

---

### 4. **API Key Setup Timing**
**What:** Tavily API key needed to be added, but we didn't check until research_node  
**Impact:**
- Could have set it up earlier
- Not critical, but could streamline

**Improvement:** Add checklist: "Verify all API keys exist in API_KEYS.env before implementing nodes that need them"

---

## What Didn't Work or Was Confusing

### 1. **Import Path Issues (Minor)**
**What:** Test file couldn't import config/nodes initially  
**Impact:**
- Had to fix import paths (adding sys.path)
- Minor friction

**Why it happened:** Guide mentioned this, but we didn't check test file structure first  
**Fix:** Guide already mentions this in "Common Gotchas" - this is fine, just caught it early.

---

### 2. **Node Dependencies Clarity**
**What:** Guide says "setup node" before "analyze node", but we had different node types  
**Impact:**
- Slightly confusing when nodes don't match the pattern
- Had to adapt the order

**Why it happened:** Our agent structure was different (research_node vs setup_node)  
**Fix:** Guide's order is a pattern, not a rule. Adapt as needed. This is fine.

---

### 3. **Template vs LLM Decision**
**What:** Guide mentions "template-based" but doesn't explicitly say when to use templates vs LLM  
**Impact:**
- Had to discuss/debate this during scaffold
- Not a problem, but could be clearer

**Fix:** Guide is actually good here - it says "start simple, add complexity later." Our incremental approach followed this.

---

## What I Would Do Differently (If Starting Over)

### 1. **Create Folder Structure Immediately** (After scaffold)
**Rationale:**
- Clearer organization from the start
- Know where files go before writing code
- Prevents reorganizing later

**Action:** Add to checklist: "Step 3: Create complete folder structure (all directories, __init__.py files)"

---

### 2. **Install Dependencies Early** (Before any imports)
**Rationale:**
- Catch import errors immediately
- Know what's available
- Fail fast if dependencies are missing

**Action:** Add to checklist: "Step 4: Install all dependencies: `pip install -r requirements.txt`"

---

### 3. **Verify API Keys Before Implementation**
**Rationale:**
- Know what's available before coding
- Set up API keys early
- Fail fast if keys are missing

**Action:** Add to checklist: "Step 5: Verify all API keys exist in API_KEYS.env (list required keys)"

---

### 4. **Create State Schema AND Config Together**
**Rationale:**
- Config depends on state schema
- Keep related code together
- Clearer dependencies

**Action:** Combine steps: "Create state schema + config together in config.py"

---

### 5. **Test Each Node Immediately After Implementation**
**Rationale:**
- Catch issues while code is fresh
- Don't accumulate multiple issues
- Faster feedback loop

**What we did:** We tested after implementing 2 nodes, which worked but could be even tighter  
**Action:** Test each node as you implement it (update smoke test incrementally)

---

## Overall Assessment

### Strengths of Your Guidelines ⭐⭐⭐⭐⭐

1. **Incremental approach** - This was the MVP's secret weapon
2. **Smoke test pattern** - Saved significant debugging time
3. **Scaffold planning** - Prevented architectural mistakes
4. **Start simple** - Avoided over-engineering
5. **Clear state contracts** - Made debugging straightforward

### Areas for Enhancement

1. **More explicit checklist format** - Some steps are buried in paragraphs
2. **Dependency management** - Add explicit "install dependencies" step
3. **Environment setup** - Add "verify API keys" checklist item
4. **Folder structure** - Make it step 3, not implied
5. **Test incrementally** - Test each node as you build it (tighter loop)

---

## Suggested Improvements to Guide

### 1. **Add "Quick Start Checklist" Section**
```
□ Read PROJECT_REQUIREMENTS.md
□ Create scaffold plan (plain English)
□ Create folder structure (all directories + __init__.py)
□ Install dependencies: pip install -r requirements.txt
□ Verify API keys in API_KEYS.env
□ Create state schema in config.py
□ Create minimal node stubs
□ Update smoke test incrementally (test as you build)
□ Wire into LangGraph after smoke test passes
```

### 2. **Add "Environment Setup Verification" Section**
- Check venv activation: `which python` should show `.venv/bin/python`
- Verify API keys: List all required keys upfront
- Check dependencies: `pip list | grep <package>`

### 3. **Clarify "Implementation Order"**
- Make it clear this is a pattern, not a rule
- Emphasize: "Simplest → Most Complex"
- Note: Adapt based on your specific node types

### 4. **Add "Test Incrementally" Pattern**
- Test each node as you implement it
- Update smoke test after each node
- Don't wait until all nodes are done

---

## Key Insights

### What Made This Build Fast and Efficient

1. **Scaffold planning** - 90% of decisions made before coding
2. **Incremental approach** - Could test orchestration separately from data quality
3. **Smoke test** - Caught issues early, before graph complexity
4. **Clear state contracts** - Each node knew exactly what to expect
5. **Dummy data first** - Got end-to-end flow working, then improved data quality

### What Made This Build Smooth

1. **No over-engineering** - Started simple, added complexity only when needed
2. **Clear error handling** - Defined strategies upfront
3. **Good logging** - Easy to see what each node was doing
4. **Type safety** - TypedDict caught issues early

---

## Final Verdict

**Your guidelines were EXCELLENT** for building this agent quickly and efficiently.

### Rating: 9/10

**What worked:**
- Incremental approach (MVP with dummy data)
- Smoke test pattern
- Scaffold planning
- Start simple principle
- Clear state contracts

**What could be improved:**
- Make checklist format more explicit
- Add dependency installation step
- Add API key verification step
- Test each node incrementally (tighter loop)

**Overall:** The guidelines helped us build a complete, working agent efficiently. The incremental approach was the MVP's secret weapon. With minor additions (checklist format, dependency management), it would be even better.

---

## Recommendation

**Update your guide with:**
1. Quick Start Checklist (1-page format)
2. Environment Setup Verification section
3. Test Incrementally pattern (test each node as you build)
4. Dependency Management section (install early, verify before imports)

**Keep these sections (they're perfect):**
- Scaffold planning
- Smoke test pattern
- Incremental approach
- Start simple principle
- State schema patterns

---

*This reflection captured after successfully building the B2B Sales Orchestrator Agent MVP.*

